# EDDK Validation Data

This notebook is used to process the validation data obtained from the Cologne Bonn Airport and:

1. generate the *flights EDDK.csv* file
2. generate the *noise events EDDK.csv* file
3. obtain the noise thresholds used at the airport

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

input_folder = Path("data/EDDK 2019")
output_folder = Path("out/eddk")

output_folder.mkdir(parents=True, exist_ok=True)

## Flights from Logs

In [ ]:
path_icao_aircraft = input_folder / "icao codes.csv"
path_arrivals = input_folder / "arrivals flightplan.csv"
path_departures = input_folder / "departures flightplan.csv"

path_out = output_folder / "EDDK flights.csv"
cols = [
    "time",
    "operation",
    "runway",
    "aircraft",
]

arr = pd.read_csv(path_arrivals, usecols=cols)
arr["time"] = pd.to_datetime(arr["time"], format=r"%d.%m.%Y %H:%M")
arr["operation"] = arr["operation"].replace("Landung", "Arrival")

dep = pd.read_csv(path_departures, usecols=cols)
dep["time"] = pd.to_datetime(dep["time"], format=r"%d.%m.%Y %H:%M")
dep["operation"] = dep["operation"].replace("Start", "Departure")

flights = pd.concat([arr, dep])

print(f"Total arrivals: {len(arr)}")
print(f"Total departures: {len(dep)}")
print(f"Total flights: {len(flights)}")

icao_acft = pd.read_csv(path_icao_aircraft)

flights = flights.merge(icao_acft[["aircraft", "icao_code"]], how="left", on="aircraft")
flights = flights.loc[flights["aircraft"].isin(icao_acft["aircraft"].loc[icao_acft["anp"] == 1])]
flights.to_csv(path_out, index=False)

print(f"Filtered arrivals: {len(flights.loc[flights['operation'] == 'Arrival'])}")
print(f"Filtered departures: {len(flights.loc[flights['operation'] == 'Departure'])}")
print(f"Filtered flights: {len(flights)}")

##  Flights from Radar

In [ ]:
path_arrivals = input_folder / "arrivals radar operations.csv"
path_departures = input_folder / "departures radar operations.csv"
path_out = output_folder / "EDDK radar operations.csv"

cols = [
    "id",
    "time",
    "operation",
    "runway",
    "aircraft",
    "plot_count",
]

arr = pd.read_csv(path_arrivals, usecols=cols)
arr_str = arr.select_dtypes("object")
arr[arr_str.columns] = arr_str.apply(lambda x: x.str.strip())
arr["time"] = pd.to_datetime(arr["time"], format=r"%d.%m.%y %H:%M:%S")
arr["operation"] = arr["operation"].replace("ARR", "Arrival")
dep = pd.read_csv(path_departures, usecols=cols)
dep_str = dep.select_dtypes("object")
dep[arr_str.columns] = dep_str.apply(lambda x: x.str.strip())
dep["time"] = pd.to_datetime(dep["time"], format=r"%d.%m.%y %H:%M:%S")
dep["operation"] = dep["operation"].replace("DEP", "Departure")
flights = pd.concat([arr, dep])
print(f"Total arrivals: {len(arr)}")
print(f"Total departures: {len(dep)}")
print(f"Total flights: {len(flights)}")

icao_acft = pd.read_csv(path_icao_aircraft)
flights = flights.merge(icao_acft[["aircraft", "icao_code"]], how="left", on="aircraft")
flights = flights.loc[flights["aircraft"].isin(icao_acft["aircraft"].loc[icao_acft["anp"] == 1])]
flights = flights.loc[flights["plot_count"] > 10].drop(columns="plot_count")
flights.to_csv(path_out, index=False)
print(f"Filtered arrivals: {len(flights.loc[flights['operation'] == 'Arrival'])}")
print(f"Filtered departures: {len(flights.loc[flights['operation'] == 'Departure'])}")
print(f"Filtered flights: {len(flights)}")

## Noise Events

In [ ]:
path_arrivals = input_folder / "arrivals noise.csv"
path_departures = input_folder / "departures noise.csv"
path_out = output_folder / "EDDK noise events.csv"
cols = [
    "id",
    "receptor",
    "time",
    "operation",
    "runway",
    "aircraft",
    "lamax",
    "sel",
]


arr = pd.read_csv(path_arrivals, usecols=cols)
arr["time"] = pd.to_datetime(arr["time"], format=r"%d.%m.%Y %H:%M")
arr["operation"] = arr["operation"].replace("Landung", "Arrival")
dep = pd.read_csv(path_departures, usecols=cols)
dep["time"] = pd.to_datetime(dep["time"], format=r"%d.%m.%Y %H:%M")
dep["operation"] = dep["operation"].replace("Start", "Departure")
noise_events = pd.concat([arr, dep])
print(f"Total arrivals: {len(arr['id'].drop_duplicates())}")
print(f"Total departures: {len(dep['id'].drop_duplicates())}")
print(f"Total flights: {len(noise_events['id'].drop_duplicates())}")

icao_acft = pd.read_csv(path_icao_aircraft)
noise_events = noise_events.loc[noise_events["aircraft"].isin(icao_acft["aircraft"].loc[icao_acft["anp"] == 1])]
noise_events = noise_events.merge(icao_acft[["aircraft", "icao_code"]], how="left", on="aircraft")

noise_events["receptor"] = noise_events["receptor"].str.replace(" ", "")
noise_events = noise_events.rename(columns={"receptor": "receptor_id"})[
    [
        "id",
        "time",
        "operation",
        "runway",
        "aircraft",
        "icao_code",
        "receptor_id",
        "lamax",
        "sel",
    ]
]
noise_events.to_csv(path_out, index=False)
print(f"Filtered arrivals: {len(noise_events.loc[noise_events['operation'] == 'Arrival', 'id'].drop_duplicates())}")
print(f"Filtered departures: {len(noise_events.loc[noise_events['operation'] == 'Departure', 'id'].drop_duplicates())}")
print(f"Filtered flights: {len(noise_events['id'].drop_duplicates())}")

## Noise Thresholds

In [ ]:
thresholds = (
    noise_events[["receptor_id", "time", "lamax"]]
    .groupby([noise_events["receptor_id"], noise_events["time"].dt.hour])
    .min()
)
thresholds["lamax"] = np.floor(thresholds["lamax"])
print(thresholds)